In [1]:
Sys.setlocale(category = "LC_ALL", locale = "en_US.UTF-8")

[1] "en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/C"

# Dependencies

In [2]:
pacman::p_load(dplyr, ggplot2, ggbeeswarm, ggpubr, ggsignif, cowplot)

# Functions

In [3]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/Misc_functions.R")

# Paths

In [4]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [5]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/"

# Main

# Meta_div with Tirp scores

In [6]:
# meta_div <- readr::read_rds(paste0(datapath, "capTCRseq/meta_div_goodsamples.rds"))

In [7]:
meta_div <- readr::read_rds(paste0(datapath, "meta_div_goodsamples_TiRP.rds"))

In [10]:
meta_div[ meta_div$Patient == "CHP_437",]

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,observed_Simpson,estimated_Simpson,SE_Simpson,basename,file,sampletype,Group,log10shann,aveTiRP,medTiRP
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
891,CHP_437_01,CHP_437,X01,0,0,9,OS,0,0,NA,...,NA,NA,NA,NA,NA,PBMC,CHP,NA,NA,NA
892,CHP_437_02,CHP_437,X02,0,0,9,OS,0,0,NA,...,NA,NA,NA,NA,NA,PBMC,CHP,NA,NA,NA
893,CHP_437_03,CHP_437,X03,0,0,9,OS,0,0,NA,...,NA,NA,NA,NA,NA,PBMC,CHP,NA,NA,NA
894,CHP_437_04,CHP_437,X04,0,0,9,OS,0,0,NA,...,NA,NA,NA,NA,NA,PBMC,CHP,NA,NA,NA
895,CHP_437_05,CHP_437,X05,0,0,9,OS,0,0,NA,...,NA,NA,NA,NA,NA,PBMC,CHP,NA,NA,NA


In [11]:
pbmc <- meta_div[ meta_div$sampletype == "PBMC",]
cfdna <- meta_div[ meta_div$sampletype == "cfDNA",]

# PBMC metrics

Add HLA

In [12]:
pca <- readr::read_rds(paste0(datapath, "/HLA/INT_HLA_PCA_postqc.rds"))

In [13]:
rownames(pca) <- paste0("CHP_", rownames(pca))

In [14]:
rownames(pca) <- gsub("-.*", "", rownames(pca))

In [15]:
pbmc$HLA_PC1 <- pca[ match(pbmc$Patient,  rownames(pca)) , "PC1"]
pbmc$HLA_PC2 <- pca[ match(pbmc$Patient,  rownames(pca)) , "PC2"]
pbmc$HLA_PC3 <- pca[ match(pbmc$Patient,  rownames(pca)) , "PC3"]

In [16]:
head(pbmc)

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,basename,file,sampletype,Group,log10shann,aveTiRP,medTiRP,HLA_PC1,HLA_PC2,HLA_PC3
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
287,CHP_10YO_M_00,CHP_10YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP_10YO-M_PBMC_DNA,batchNormal_16Apr2020/mixcr/clones/CLONES_TRBCHP_10YO-M_PBMC_DNA.txt,PBMC,CHP,1.0249370,0.3150243,0.2722006,NA,NA,NA
288,CHP_11YO_M_00,CHP_11YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP_11YO-M_PBMC_DNA,batchNormal_16Apr2020/mixcr/clones/CLONES_TRBCHP_11YO-M_PBMC_DNA.txt,PBMC,CHP,1.0647200,0.2588929,0.3772438,NA,NA,NA
289,CHP_15YO_F_A_00,CHP_15YO_F_A,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP_15YO-F-A_PBMC_DNA,batchNormal_16Apr2020/mixcr/clones/CLONES_TRBCHP_15YO-F-A_PBMC_DNA.txt,PBMC,CHP,0.9533731,0.4688676,0.5159322,NA,NA,NA
290,CHP_15YO_F_B_00,CHP_15YO_F_B,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP_15YO-F-B_PBMC_DNA,batchNormal_16Apr2020/mixcr/clones/CLONES_TRBCHP_15YO-F-B_PBMC_DNA.txt,PBMC,CHP,0.8014037,-0.1303953,0.2653686,NA,NA,NA
291,CHP_15YO_M_00,CHP_15YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP_15YO-M_PBMC_DNA,batchNormal_16Apr2020/mixcr/clones/CLONES_TRBCHP_15YO-M_PBMC_DNA.txt,PBMC,CHP,1.3244882,0.5822285,0.9216413,NA,NA,NA
292,CHP_16YO_M_00,CHP_16YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP_16YO-M_PBMC_DNA,batchNormal_16Apr2020/mixcr/clones/CLONES_TRBCHP_16YO-M_PBMC_DNA.txt,PBMC,CHP,0.5990092,0.2910802,0.6214764,NA,NA,NA


Add V PCA

In [17]:
v_pca <- readr::read_rds( paste0(datapath, "/capTCRseq/INT_PBMC_V_PCA_postqc.rds"))

In [18]:
pbmc$V_PC1 <- v_pca[ match(pbmc$basename,  rownames(v_pca)) , "PC1"]
pbmc$V_PC2 <- v_pca[ match(pbmc$basename,  rownames(v_pca)) , "PC2"]
pbmc$V_PC3 <- v_pca[ match(pbmc$basename,  rownames(v_pca)) , "PC3"]

In [19]:
head(pbmc[!is.na(pbmc$V_PC1),])

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,Group,log10shann,aveTiRP,medTiRP,HLA_PC1,HLA_PC2,HLA_PC3,V_PC1,V_PC2,V_PC3
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
287,CHP_10YO_M_00,CHP_10YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP,1.0249370,0.3150243,0.2722006,NA,NA,NA,4.487279,0.3424950,-0.3658488
288,CHP_11YO_M_00,CHP_11YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP,1.0647200,0.2588929,0.3772438,NA,NA,NA,4.608904,0.2220972,-0.2893333
289,CHP_15YO_F_A_00,CHP_15YO_F_A,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP,0.9533731,0.4688676,0.5159322,NA,NA,NA,4.672933,0.2820477,-0.2893257
290,CHP_15YO_F_B_00,CHP_15YO_F_B,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP,0.8014037,-0.1303953,0.2653686,NA,NA,NA,4.695359,0.1389736,-0.1555102
291,CHP_15YO_M_00,CHP_15YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP,1.3244882,0.5822285,0.9216413,NA,NA,NA,4.374467,0.1322533,-0.3353389
292,CHP_16YO_M_00,CHP_16YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,CHP,0.5990092,0.2910802,0.6214764,NA,NA,NA,4.792346,0.2664800,-0.1734509


Add J PCA

In [20]:
j_pca <- readr::read_rds(paste0(datapath, "/capTCRseq/INT_PBMC_J_PCA_postqc.rds"))

In [21]:
pbmc$J_PC1 <- j_pca[ match(pbmc$basename,  rownames(j_pca)) , "PC1"]
pbmc$J_PC2 <- j_pca[ match(pbmc$basename,  rownames(j_pca)) , "PC2"]
pbmc$J_PC3 <- j_pca[ match(pbmc$basename,  rownames(j_pca)) , "PC3"]

Add GLIPH metrics

In [22]:
h4hpath

[1] "/Users/anabbi/Desktop/H4H/INTERCEPT/"

In [25]:
gliph_out <- readr::read_csv(paste0(h4hpath, "analysis/GLIPH/run_gliph/v5/gliph_output/int_vdj_v5_cluster.csv"), col_names = T) #  gliph reference dataset  is MItechel

Rows: 98847 Columns: 29
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (17): pattern, type, TcRb, V, J, Sample, HLA-A, HLA-B, HLA-C, HLA-DPA1, ...
dbl (11): index, Fisher_score, number_subject, number_unique_cdr3, final_sco...
lgl  (1): TcRa

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [26]:
# remove columns starting with HLA
gliph_out <- gliph_out[,!grepl("HLA", colnames(gliph_out))]

In [27]:
tail(gliph_out)

index,pattern,Fisher_score,number_subject,number_unique_cdr3,final_score,hla_score,vb_score,expansion_score,length_score,cluster_size_score,type,TcRb,V,J,TcRa,Sample,Freq
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<dbl>
59927,single,1,1,1,2.8e-07,1,0.001,0.29,0.001,0.95,global-SPRGTSGGAGDNE,CASSPRGTSGGAGDNEQFF,TRBV7-2,TRBJ2-1,NA,Leukemia:CHP_392_X04-PBMC,123
59927,single,1,1,1,2.8e-07,1,0.001,0.29,0.001,0.95,global-SPRGTSGGAGDNE,CASSPRGTSGGAGDNEQFF,TRBV7-2,TRBJ2-1,NA,Leukemia:CHP_392_X04-PBMC,8
59972,single,1,1,1,7.8e-07,1,0.001,0.82,0.001,0.95,global-SIPPTGVG,CASSIPPTGVGYTF,TRBV28,TRBJ1-2,NA,Solidtumors:CHP_364_X01-PBMC,7
59972,single,1,1,1,7.8e-07,1,0.001,0.82,0.001,0.95,global-SIPPTGVG,CASSIPPTGVGYTF,TRBV28,TRBJ1-2,NA,Solidtumors:CHP_364_X01-PBMC,1
60011,single,1,1,1,1.6e-07,1,0.001,0.17,0.001,0.95,global-SQDQISGNT,CASSQDQISGNTIYF,TRBV3-1,TRBJ1-3,NA,Leukemia:CHP_396_X04-PBMC,222
60011,single,1,1,1,1.6e-07,1,0.001,0.17,0.001,0.95,global-SQDQISGNT,CASSQDQISGNTIYF,TRBV3-1,TRBJ1-3,NA,Leukemia:CHP_396_X04-PBMC,30


In [28]:
dim(gliph_out)

[1] 98847    18

In [29]:
gliph_out <- gliph_out[gliph_out$pattern != "single",]
gliph_out <- gliph_out[gliph_out$number_unique_cdr3 >= 3,]

In [30]:
dim(gliph_out)

[1] 65202    18

In [31]:
mytab <- as.data.frame.matrix(table(gliph_out$Sample, gliph_out$pattern) )

In [32]:
mytab <- mytab[ rowSums(mytab) > 0,]

In [33]:
tmp1 <- rowSums(mytab != 0) # count non zeros = number of groups

In [34]:
tail(tmp1)

T-cellmalignancies:CHP_387_X01-PBMC T-cellmalignancies:CHP_387_X02-cfDNA 
                                 146                                    5 
T-cellmalignancies:CHP_387_X03-cfDNA  T-cellmalignancies:CHP_387_X03-PBMC 
                                   9                                   17 
T-cellmalignancies:CHP_395_X01-cfDNA T-cellmalignancies:CHP_395_X02-cfDNA 
                                   2                                    9

In [35]:
pbmc$samplename <- paste0(pbmc$cancergroup, ":", pbmc$Patient, "_", pbmc$cycle, "-", pbmc$sampletype)

In [36]:
pbmc$samplename <- gsub(" ", "", pbmc$samplename)

In [37]:
# add column gliph_n to meta_div, match meta_div$samplename to names(tmp1)
pbmc$gliph_n <- tmp1[match(pbmc$samplename, names(tmp1))]

In [38]:
head(pbmc)

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,HLA_PC2,HLA_PC3,V_PC1,V_PC2,V_PC3,J_PC1,J_PC2,J_PC3,samplename,gliph_n
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
287,CHP_10YO_M_00,CHP_10YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.487279,0.3424950,-0.3658488,2.697180,0.2614244,-0.083404572,Normal:CHP_10YO_M_X00-PBMC,9
288,CHP_11YO_M_00,CHP_11YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.608904,0.2220972,-0.2893333,2.623844,0.2712649,-0.009213308,Normal:CHP_11YO_M_X00-PBMC,3
289,CHP_15YO_F_A_00,CHP_15YO_F_A,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.672933,0.2820477,-0.2893257,2.715678,0.2214375,-0.022667262,Normal:CHP_15YO_F_A_X00-PBMC,7
290,CHP_15YO_F_B_00,CHP_15YO_F_B,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.695359,0.1389736,-0.1555102,2.735124,0.2395413,-0.085880490,Normal:CHP_15YO_F_B_X00-PBMC,1
291,CHP_15YO_M_00,CHP_15YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.374467,0.1322533,-0.3353389,2.583896,0.2814897,-0.023352703,Normal:CHP_15YO_M_X00-PBMC,12
292,CHP_16YO_M_00,CHP_16YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.792346,0.2664800,-0.1734509,2.735426,-0.0465748,-0.167060816,Normal:CHP_16YO_M_X00-PBMC,5


In [39]:
pbmc$gliph_n[is.na(pbmc$gliph_n)] <- 0

In [40]:
gliphdf <- data.frame(gliph = unique(gliph_out$pattern))
rownames(gliphdf) <- gliphdf$gliph
# gliphdf$Class <- onetest$hklust[match(rownames(gliphdf), onetest$Xpattern)]

In [41]:
head(gliphdf)

,gliph
,<chr>
SLG%E,SLG%E
S%GGNTE,S%GGNTE
GIWV,GIWV
IWV,IWV
WVYE,WVYE
IWVY,IWVY


In [42]:
gliph_out$subject <- gsub(":.*", "", gliph_out$Sample)

In [118]:
for(i in 1:nrow(gliphdf)){
mygliph <- gliph_out[gliph_out$pattern == gliphdf$gliph[i],]
# add 1 in gliphdf$CMV, if mygliph$subject ciontains CMV
gliphdf$CMV[i] <- ifelse(sum(grepl("CMV", mygliph$subject)) > 0, 1, 0)
gliphdf$EBV[i] <- ifelse(sum(grepl("EBV", mygliph$subject)) > 0, 1, 0)
gliphdf$HCV[i] <- ifelse(sum(grepl("HCV", mygliph$subject)) > 0, 1, 0)
gliphdf$Flu[i] <- ifelse(sum(grepl("Influenza", mygliph$subject)) > 0, 1, 0)
gliphdf$hs[i] <- ifelse(sum(grepl("HomoSapiens", mygliph$subject)) > 0, 1, 0)
}

In [119]:
tail(gliphdf)

,gliph,CMV,EBV,HCV,Flu,hs
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SP%,SP%,0,0,0,0,0
S%E,S%E,0,0,0,0,0
S%GTG,S%GTG,0,0,0,0,0
G%,G%,0,0,0,0,1
%G,%G,0,0,0,0,0
%E,%E,0,0,0,0,0


In [120]:
head(pbmc)

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,HLA_PC2,HLA_PC3,V_PC1,V_PC2,V_PC3,J_PC1,J_PC2,J_PC3,samplename,gliph_n
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
287,CHP_10YO_M_00,CHP_10YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.487279,0.3424950,-0.3658488,2.697180,0.2614244,-0.083404572,Normal:CHP_10YO_M_X00-PBMC,15
288,CHP_11YO_M_00,CHP_11YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.608904,0.2220972,-0.2893333,2.623844,0.2712649,-0.009213308,Normal:CHP_11YO_M_X00-PBMC,1
289,CHP_15YO_F_A_00,CHP_15YO_F_A,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.672933,0.2820477,-0.2893257,2.715678,0.2214375,-0.022667262,Normal:CHP_15YO_F_A_X00-PBMC,7
290,CHP_15YO_F_B_00,CHP_15YO_F_B,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.695359,0.1389736,-0.1555102,2.735124,0.2395413,-0.085880490,Normal:CHP_15YO_F_B_X00-PBMC,3
291,CHP_15YO_M_00,CHP_15YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.374467,0.1322533,-0.3353389,2.583896,0.2814897,-0.023352703,Normal:CHP_15YO_M_X00-PBMC,14
292,CHP_16YO_M_00,CHP_16YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,NA,NA,4.792346,0.2664800,-0.1734509,2.735426,-0.0465748,-0.167060816,Normal:CHP_16YO_M_X00-PBMC,4


In [121]:
pbmc$CMV <- NA
pbmc$EBV <- NA
pbmc$HCV <- NA
pbmc$Flu <- NA
pbmc$hs <- NA

In [122]:
for(i in 1:nrow(pbmc) ){
    mygliph <- gliph_out[ gliph_out$Sample == pbmc$samplename[i], ]
    tmp <- gliphdf[ match(mygliph$pattern, gliphdf$gliph),]
    vsums <- colSums(tmp[,2:6])
    pbmc[i, c("CMV", "EBV", "HCV", "Flu", "hs")] <- vsums
}

In [123]:
head(pbmc)

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,J_PC1,J_PC2,J_PC3,samplename,gliph_n,CMV,EBV,HCV,Flu,hs
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
287,CHP_10YO_M_00,CHP_10YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,2.697180,0.2614244,-0.083404572,Normal:CHP_10YO_M_X00-PBMC,15,4,1,0,0,1
288,CHP_11YO_M_00,CHP_11YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,2.623844,0.2712649,-0.009213308,Normal:CHP_11YO_M_X00-PBMC,1,0,0,0,0,0
289,CHP_15YO_F_A_00,CHP_15YO_F_A,X00,NA,NA,NA,Normal,NA,NA,NA,...,2.715678,0.2214375,-0.022667262,Normal:CHP_15YO_F_A_X00-PBMC,7,0,0,0,0,0
290,CHP_15YO_F_B_00,CHP_15YO_F_B,X00,NA,NA,NA,Normal,NA,NA,NA,...,2.735124,0.2395413,-0.085880490,Normal:CHP_15YO_F_B_X00-PBMC,3,1,0,0,0,0
291,CHP_15YO_M_00,CHP_15YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,2.583896,0.2814897,-0.023352703,Normal:CHP_15YO_M_X00-PBMC,14,0,1,0,0,0
292,CHP_16YO_M_00,CHP_16YO_M,X00,NA,NA,NA,Normal,NA,NA,NA,...,2.735426,-0.0465748,-0.167060816,Normal:CHP_16YO_M_X00-PBMC,4,0,1,0,0,0


# cfDNA

V gene

In [124]:
v_cfdna_pca <- readr::read_rds(paste0(datapath, "/capTCRseq/INT_cfDNA_V_PCA_postqc.rds"))

In [125]:
cfdna$V_PC1 <- v_cfdna_pca[ match(cfdna$basename,  rownames(v_cfdna_pca)) , "PC1"]
cfdna$V_PC2 <- v_cfdna_pca[ match(cfdna$basename,  rownames(v_cfdna_pca)) , "PC2"]
cfdna$V_PC3 <- v_cfdna_pca[ match(cfdna$basename,  rownames(v_cfdna_pca)) , "PC3"]

In [126]:
head(cfdna)

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,basename,file,sampletype,Group,log10shann,aveTiRP,medTiRP,V_PC1,V_PC2,V_PC3
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CHP_323_03,CHP_323,X03,0,786.24,1,HR ALL,0,0,NA,...,CHP_323-03_cfDNA,batch3a9_cfDNA_10Jul2020/mixcr/clones/CLONES_TRBCHP_323-03_cfDNA.txt,cfDNA,CHP,1.6377198,0.04826843,0.1393051,15.83866850,7.9531074,9.4381691
2,CHP_323_05,CHP_323,X05,0,979.50,1,HR ALL,0,0,NA,...,CHP_323-05_cfDNA,remaining_batch_1_cfDNA_8April2021/mixcr/clones/CLONES_TRBCHP_323-05_cfDNA.txt,cfDNA,CHP,0.9380692,0.25983842,0.5856662,0.03694791,0.2037428,0.9895571
3,CHP_324_04,CHP_324,X04,0,0.00,12,T-ALL,0,0,NA,...,CHP_324-04_cfDNA,batch5_cfDNA_25Jun2020/mixcr/clones/CLONES_TRBCHP_324-04_cfDNA.txt,cfDNA,CHP,0.0000000,-0.85235289,-0.8523529,-2.42588499,-0.2790740,0.1117550
4,CHP_324_05,CHP_324,X05,0,0.00,12,T-ALL,0,0,NA,...,CHP_324-05_cfDNA,batch10_cfDNA_13Jul2020/mixcr/clones/CLONES_TRBCHP_324-05_cfDNA.txt,cfDNA,CHP,0.2491984,-1.17906074,-1.1790607,-2.20171492,-0.2392266,0.1373460
5,CHP_326_05,CHP_326,X05,0,972.90,4,SR ALL,0,0,NA,...,CHP_326-05_cfDNA,batch10_cfDNA_13Jul2020/mixcr/clones/CLONES_TRBCHP_326-05_cfDNA.txt,cfDNA,CHP,1.2165354,0.08628995,0.2607004,1.53369308,0.6800554,-0.7668898
6,CHP_328_05,CHP_328,X05,0,91.16,7,SR ALL,0,0,NA,...,CHP_328-05_cfDNA,batch10_cfDNA_13Jul2020/mixcr/clones/CLONES_TRBCHP_328-05_cfDNA.txt,cfDNA,CHP,0.4864305,NA,NA,-1.73881538,-0.8087552,-0.8391630


J gene

In [127]:
j_pca <- readr::read_rds(paste0(datapath, "/capTCRseq/INT_cfDNA_J_PCA_postqc.rds"))

In [128]:
cfdna$J_PC1 <- j_pca[ match(cfdna$basename,  rownames(j_pca)) , "PC1"]
cfdna$J_PC2 <- j_pca[ match(cfdna$basename,  rownames(j_pca)) , "PC2"]
cfdna$J_PC3 <- j_pca[ match(cfdna$basename,  rownames(j_pca)) , "PC3"]

GLIPH

In [129]:
gliph_out <- readr::read_csv(paste0(h4hpath, "analysis/GLIPH/run_gliph/v3_refMitchel/int_vdj_v2_ref_Mitchel_cluster.csv"), col_names = T)

Rows: 42815 Columns: 29
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (17): pattern, type, TcRb, V, J, Sample, HLA-A, HLA-B, HLA-C, HLA-DPA1, ...
dbl (11): index, Fisher_score, number_subject, number_unique_cdr3, final_sco...
lgl  (1): TcRa

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [130]:
# remove columns starting with HLA
gliph_out <- gliph_out[,!grepl("HLA", colnames(gliph_out))]

In [131]:
tail(gliph_out)

index,pattern,Fisher_score,number_subject,number_unique_cdr3,final_score,hla_score,vb_score,expansion_score,length_score,cluster_size_score,type,TcRb,V,J,TcRa,Sample,Freq
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<dbl>
47424,single,1,1,1,4.1e-07,1,0.001,0.43,0.001,0.95,global-SPRGTSGGAGDNE,CASSPRGTSGGAGDNEQFF,TRBV7-2,TRBJ2-1,NA,Leukemia:CHP_392_X04-PBMC,123
47424,single,1,1,1,4.1e-07,1,0.001,0.43,0.001,0.95,global-SPRGTSGGAGDNE,CASSPRGTSGGAGDNEQFF,TRBV7-2,TRBJ2-1,NA,Leukemia:CHP_392_X04-PBMC,8
47472,single,1,1,1,9.1e-07,1,0.001,0.95,0.001,0.95,global-SIPPTGVG,CASSIPPTGVGYTF,TRBV28,TRBJ1-2,NA,Solidtumors:CHP_364_X01-PBMC,7
47472,single,1,1,1,9.1e-07,1,0.001,0.95,0.001,0.95,global-SIPPTGVG,CASSIPPTGVGYTF,TRBV28,TRBJ1-2,NA,Solidtumors:CHP_364_X01-PBMC,1
47512,single,1,1,1,2.4e-07,1,0.001,0.25,0.001,0.95,global-SQDQISGNT,CASSQDQISGNTIYF,TRBV3-1,TRBJ1-3,NA,Leukemia:CHP_396_X04-PBMC,222
47512,single,1,1,1,2.4e-07,1,0.001,0.25,0.001,0.95,global-SQDQISGNT,CASSQDQISGNTIYF,TRBV3-1,TRBJ1-3,NA,Leukemia:CHP_396_X04-PBMC,30


In [132]:
dim(gliph_out)

[1] 42815    18

In [133]:
gliph_out <- gliph_out[gliph_out$pattern != "single",]

In [134]:
dim(gliph_out)

[1] 40384    18

In [135]:
mytab <- as.data.frame.matrix(table(gliph_out$Sample, gliph_out$pattern) )

In [136]:
mytab <- mytab[ rowSums(mytab) > 0,]

In [137]:
tmp1 <- rowSums(mytab != 0) # count non zeros = number of groups

In [138]:
tail(tmp1)

T-cellmalignancies:CHP_387_X01-PBMC T-cellmalignancies:CHP_387_X02-cfDNA 
                                 204                                    8 
T-cellmalignancies:CHP_387_X03-cfDNA  T-cellmalignancies:CHP_387_X03-PBMC 
                                  13                                   32 
T-cellmalignancies:CHP_395_X01-cfDNA T-cellmalignancies:CHP_395_X02-cfDNA 
                                   4                                   14

In [139]:
cfdna$samplename <- paste0(cfdna$cancergroup, ":", cfdna$Patient, "_", cfdna$cycle, "-", cfdna$sampletype)

In [140]:
cfdna$samplename <- gsub(" ", "", cfdna$samplename)

In [141]:
# add column gliph_n to meta_div, match meta_div$samplename to names(tmp1)
cfdna$gliph_n <- tmp1[match(cfdna$samplename, names(tmp1))]

In [142]:
head(cfdna)

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,aveTiRP,medTiRP,V_PC1,V_PC2,V_PC3,J_PC1,J_PC2,J_PC3,samplename,gliph_n
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,CHP_323_03,CHP_323,X03,0,786.24,1,HR ALL,0,0,NA,...,0.04826843,0.1393051,15.83866850,7.9531074,9.4381691,-12.2473929,-1.42025844,-1.04697016,Leukemia:CHP_323_X03-cfDNA,71
2,CHP_323_05,CHP_323,X05,0,979.50,1,HR ALL,0,0,NA,...,0.25983842,0.5856662,0.03694791,0.2037428,0.9895571,-0.2918142,-1.03382807,0.68828218,Leukemia:CHP_323_X05-cfDNA,9
3,CHP_324_04,CHP_324,X04,0,0.00,12,T-ALL,0,0,NA,...,-0.85235289,-0.8523529,-2.42588499,-0.2790740,0.1117550,1.7122042,0.09042876,-0.07326199,T-cellmalignancies:CHP_324_X04-cfDNA,1
4,CHP_324_05,CHP_324,X05,0,0.00,12,T-ALL,0,0,NA,...,-1.17906074,-1.1790607,-2.20171492,-0.2392266,0.1373460,1.5349325,0.14012792,-0.08421036,T-cellmalignancies:CHP_324_X05-cfDNA,2
5,CHP_326_05,CHP_326,X05,0,972.90,4,SR ALL,0,0,NA,...,0.08628995,0.2607004,1.53369308,0.6800554,-0.7668898,-0.9056822,0.51843452,-0.90888221,Leukemia:CHP_326_X05-cfDNA,13
6,CHP_328_05,CHP_328,X05,0,91.16,7,SR ALL,0,0,NA,...,NA,NA,-1.73881538,-0.8087552,-0.8391630,1.2267221,0.09263065,0.06769804,Leukemia:CHP_328_X05-cfDNA,NA


In [143]:
cfdna$gliph_n[is.na(cfdna$gliph_n)] <- 0

In [144]:
gliphdf <- data.frame(gliph = unique(gliph_out$pattern))
rownames(gliphdf) <- gliphdf$gliph

In [145]:
head(gliphdf)

,gliph
,<chr>
GIWV,GIWV
IWV,IWV
WVYE,WVYE
IWVY,IWVY
WVY,WVY
TGIW,TGIW


In [146]:
gliph_out$subject <- gsub(":.*", "", gliph_out$Sample)

In [147]:
for(i in 1:nrow(gliphdf)){
mygliph <- gliph_out[gliph_out$pattern == gliphdf$gliph[i],]
# add 1 in gliphdf$CMV, if mygliph$subject ciontains CMV
gliphdf$CMV[i] <- ifelse(sum(grepl("CMV", mygliph$subject)) > 0, 1, 0)
gliphdf$EBV[i] <- ifelse(sum(grepl("EBV", mygliph$subject)) > 0, 1, 0)
gliphdf$HCV[i] <- ifelse(sum(grepl("HCV", mygliph$subject)) > 0, 1, 0)
gliphdf$Flu[i] <- ifelse(sum(grepl("Influenza", mygliph$subject)) > 0, 1, 0)
gliphdf$hs[i] <- ifelse(sum(grepl("HomoSapiens", mygliph$subject)) > 0, 1, 0)
}

In [148]:
tail(gliphdf)

,gliph,CMV,EBV,HCV,Flu,hs
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SP%,SP%,0,0,0,0,0
S%E,S%E,0,0,0,0,0
S%GTG,S%GTG,0,0,0,0,0
G%,G%,0,0,0,0,1
%G,%G,0,0,0,0,0
%E,%E,0,0,0,0,0


In [149]:
head(cfdna)

,sample_id,Patient,cycle,FOLD,ATC,Age,Disease_type,TE_fold,CD3,EM,...,aveTiRP,medTiRP,V_PC1,V_PC2,V_PC3,J_PC1,J_PC2,J_PC3,samplename,gliph_n
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<int>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,CHP_323_03,CHP_323,X03,0,786.24,1,HR ALL,0,0,NA,...,0.04826843,0.1393051,15.83866850,7.9531074,9.4381691,-12.2473929,-1.42025844,-1.04697016,Leukemia:CHP_323_X03-cfDNA,71
2,CHP_323_05,CHP_323,X05,0,979.50,1,HR ALL,0,0,NA,...,0.25983842,0.5856662,0.03694791,0.2037428,0.9895571,-0.2918142,-1.03382807,0.68828218,Leukemia:CHP_323_X05-cfDNA,9
3,CHP_324_04,CHP_324,X04,0,0.00,12,T-ALL,0,0,NA,...,-0.85235289,-0.8523529,-2.42588499,-0.2790740,0.1117550,1.7122042,0.09042876,-0.07326199,T-cellmalignancies:CHP_324_X04-cfDNA,1
4,CHP_324_05,CHP_324,X05,0,0.00,12,T-ALL,0,0,NA,...,-1.17906074,-1.1790607,-2.20171492,-0.2392266,0.1373460,1.5349325,0.14012792,-0.08421036,T-cellmalignancies:CHP_324_X05-cfDNA,2
5,CHP_326_05,CHP_326,X05,0,972.90,4,SR ALL,0,0,NA,...,0.08628995,0.2607004,1.53369308,0.6800554,-0.7668898,-0.9056822,0.51843452,-0.90888221,Leukemia:CHP_326_X05-cfDNA,13
6,CHP_328_05,CHP_328,X05,0,91.16,7,SR ALL,0,0,NA,...,NA,NA,-1.73881538,-0.8087552,-0.8391630,1.2267221,0.09263065,0.06769804,Leukemia:CHP_328_X05-cfDNA,0


In [150]:
cfdna$CMV <- NA
cfdna$EBV <- NA
cfdna$HCV <- NA
cfdna$Flu <- NA
cfdna$hs <- NA

In [151]:
for(i in 1:nrow(cfdna) ){
    mygliph <- gliph_out[ gliph_out$Sample == cfdna$samplename[i], ]
    tmp <- gliphdf[ match(mygliph$pattern, gliphdf$gliph),]
    vsums <- colSums(tmp[,2:6])
    cfdna[i, c("CMV", "EBV", "HCV", "Flu", "hs")] <- vsums
}

In [152]:
colnames(cfdna)

[1] "sample_id"          "Patient"            "cycle"             
 [4] "FOLD"               "ATC"                "Age"               
 [7] "Disease_type"       "TE_fold"            "CD3"               
[10] "EM"                 "CM"                 "Naïve"             
[13] "SCM"                "TE"                 "EM%"               
[16] "CM%"                "Naïve%"             "SCM%"              
[19] "TE%"                "Naïve/TE"           "N+CM+SCM/TE+EM"    
[22] "PD-1"               "PDL1"               "LAG3"              
[25] "TIM3"               "CTLA4"              "PD1%"              
[28] "PDL1%"              "LAG3%"              "TIM3%"             
[31] "CTLA4%"             "Any NCR"            "PD1 + LAG3 + TIM3" 
[34] "cancergroup"        "agegroup"           "foldgroup"         
[37] "Remission"          "Relapse"            "Transplant"        
[40] "CART"               "vital_status"       "LOF"               
[43] "TRB"                "Reads"              "CPKR"              
[46] "Average_reads"      "VMR"                "Max_reads"         
[49] "Singletons"         "Doubletons"         "qD"                
[52] "Sample_Coverage"    "observed_Richness"  "estimated_Richness"
[55] "SE_Richeness"       "observed_Shannon"   "estimated_Shannon" 
[58] "SE_Shannon"         "observed_Simpson"   "estimated_Simpson" 
[61] "SE_Simpson"         "basename"           "file"              
[64] "sampletype"         "Group"              "log10shann"        
[67] "aveTiRP"            "medTiRP"            "V_PC1"             
[70] "V_PC2"              "V_PC3"              "J_PC1"             
[73] "J_PC2"              "J_PC3"              "samplename"        
[76] "gliph_n"            "CMV"                "EBV"               
[79] "HCV"                "Flu"                "hs"

In [153]:
colnames(cfdna)[43:ncol(cfdna)] <- paste0("cfDNA_", colnames(cfdna)[43:ncol(cfdna)] )

In [154]:
colnames(cfdna)

[1] "sample_id"                "Patient"                 
 [3] "cycle"                    "FOLD"                    
 [5] "ATC"                      "Age"                     
 [7] "Disease_type"             "TE_fold"                 
 [9] "CD3"                      "EM"                      
[11] "CM"                       "Naïve"                   
[13] "SCM"                      "TE"                      
[15] "EM%"                      "CM%"                     
[17] "Naïve%"                   "SCM%"                    
[19] "TE%"                      "Naïve/TE"                
[21] "N+CM+SCM/TE+EM"           "PD-1"                    
[23] "PDL1"                     "LAG3"                    
[25] "TIM3"                     "CTLA4"                   
[27] "PD1%"                     "PDL1%"                   
[29] "LAG3%"                    "TIM3%"                   
[31] "CTLA4%"                   "Any NCR"                 
[33] "PD1 + LAG3 + TIM3"        "cancergroup"             
[35] "agegroup"                 "foldgroup"               
[37] "Remission"                "Relapse"                 
[39] "Transplant"               "CART"                    
[41] "vital_status"             "LOF"                     
[43] "cfDNA_TRB"                "cfDNA_Reads"             
[45] "cfDNA_CPKR"               "cfDNA_Average_reads"     
[47] "cfDNA_VMR"                "cfDNA_Max_reads"         
[49] "cfDNA_Singletons"         "cfDNA_Doubletons"        
[51] "cfDNA_qD"                 "cfDNA_Sample_Coverage"   
[53] "cfDNA_observed_Richness"  "cfDNA_estimated_Richness"
[55] "cfDNA_SE_Richeness"       "cfDNA_observed_Shannon"  
[57] "cfDNA_estimated_Shannon"  "cfDNA_SE_Shannon"        
[59] "cfDNA_observed_Simpson"   "cfDNA_estimated_Simpson" 
[61] "cfDNA_SE_Simpson"         "cfDNA_basename"          
[63] "cfDNA_file"               "cfDNA_sampletype"        
[65] "cfDNA_Group"              "cfDNA_log10shann"        
[67] "cfDNA_aveTiRP"            "cfDNA_medTiRP"           
[69] "cfDNA_V_PC1"              "cfDNA_V_PC2"             
[71] "cfDNA_V_PC3"              "cfDNA_J_PC1"             
[73] "cfDNA_J_PC2"              "cfDNA_J_PC3"             
[75] "cfDNA_samplename"         "cfDNA_gliph_n"           
[77] "cfDNA_CMV"                "cfDNA_EBV"               
[79] "cfDNA_HCV"                "cfDNA_Flu"               
[81] "cfDNA_hs"

# COmbine PBMC and cfDNA

In [155]:
sample_df <- pbmc

In [156]:
sample_df <- cbind(sample_df, cfdna[ match(sample_df$sample_id, cfdna$sample_id), 43:ncol(cfdna)])

In [157]:
colnames(sample_df)

[1] "sample_id"                "Patient"                 
  [3] "cycle"                    "FOLD"                    
  [5] "ATC"                      "Age"                     
  [7] "Disease_type"             "TE_fold"                 
  [9] "CD3"                      "EM"                      
 [11] "CM"                       "Naïve"                   
 [13] "SCM"                      "TE"                      
 [15] "EM%"                      "CM%"                     
 [17] "Naïve%"                   "SCM%"                    
 [19] "TE%"                      "Naïve/TE"                
 [21] "N+CM+SCM/TE+EM"           "PD-1"                    
 [23] "PDL1"                     "LAG3"                    
 [25] "TIM3"                     "CTLA4"                   
 [27] "PD1%"                     "PDL1%"                   
 [29] "LAG3%"                    "TIM3%"                   
 [31] "CTLA4%"                   "Any NCR"                 
 [33] "PD1 + LAG3 + TIM3"        "cancergroup"             
 [35] "agegroup"                 "foldgroup"               
 [37] "Remission"                "Relapse"                 
 [39] "Transplant"               "CART"                    
 [41] "vital_status"             "LOF"                     
 [43] "TRB"                      "Reads"                   
 [45] "CPKR"                     "Average_reads"           
 [47] "VMR"                      "Max_reads"               
 [49] "Singletons"               "Doubletons"              
 [51] "qD"                       "Sample_Coverage"         
 [53] "observed_Richness"        "estimated_Richness"      
 [55] "SE_Richeness"             "observed_Shannon"        
 [57] "estimated_Shannon"        "SE_Shannon"              
 [59] "observed_Simpson"         "estimated_Simpson"       
 [61] "SE_Simpson"               "basename"                
 [63] "file"                     "sampletype"              
 [65] "Group"                    "log10shann"              
 [67] "aveTiRP"                  "medTiRP"                 
 [69] "HLA_PC1"                  "HLA_PC2"                 
 [71] "HLA_PC3"                  "V_PC1"                   
 [73] "V_PC2"                    "V_PC3"                   
 [75] "J_PC1"                    "J_PC2"                   
 [77] "J_PC3"                    "samplename"              
 [79] "gliph_n"                  "CMV"                     
 [81] "EBV"                      "HCV"                     
 [83] "Flu"                      "hs"                      
 [85] "cfDNA_TRB"                "cfDNA_Reads"             
 [87] "cfDNA_CPKR"               "cfDNA_Average_reads"     
 [89] "cfDNA_VMR"                "cfDNA_Max_reads"         
 [91] "cfDNA_Singletons"         "cfDNA_Doubletons"        
 [93] "cfDNA_qD"                 "cfDNA_Sample_Coverage"   
 [95] "cfDNA_observed_Richness"  "cfDNA_estimated_Richness"
 [97] "cfDNA_SE_Richeness"       "cfDNA_observed_Shannon"  
 [99] "cfDNA_estimated_Shannon"  "cfDNA_SE_Shannon"        
[101] "cfDNA_observed_Simpson"   "cfDNA_estimated_Simpson" 
[103] "cfDNA_SE_Simpson"         "cfDNA_basename"          
[105] "cfDNA_file"               "cfDNA_sampletype"        
[107] "cfDNA_Group"              "cfDNA_log10shann"        
[109] "cfDNA_aveTiRP"            "cfDNA_medTiRP"           
[111] "cfDNA_V_PC1"              "cfDNA_V_PC2"             
[113] "cfDNA_V_PC3"              "cfDNA_J_PC1"             
[115] "cfDNA_J_PC2"              "cfDNA_J_PC3"             
[117] "cfDNA_samplename"         "cfDNA_gliph_n"           
[119] "cfDNA_CMV"                "cfDNA_EBV"               
[121] "cfDNA_HCV"                "cfDNA_Flu"               
[123] "cfDNA_hs"

In [158]:
readr::write_rds(sample_df, paste0(datapath, "int_sample_df.rds"))